# Analisi del testo

L'elaborazione del linguaggio naturale (NLP) è un ramo dell'intelligenza artificiale (AI) che si occupa del linguaggio scritto e parlato. Puoi usare NLP per sviluppare soluzioni che estraggono il significato semantico dal testo o dal discorso, o che formulano risposte significative in linguaggio naturale.

I *servizi cognitivi* di Microsoft Azure includono il servizio *Analisi del testo*, che fornisce alcune capacità NLP predefinite, tra cui l'identificazione di frasi chiave nel testo e la classificazione del testo basata sulla valutazione.

![Un robot che legge un notebook](./images/NLP.jpg)

Ad esempio, supponiamo che l'organizzazione fittizia *Margie's Travel* incoraggi i clienti a inviare recensioni per i soggiorni in hotel. Potresti usare il servizio Analisi del testo per riassumere le recensioni estraendo le frasi chiave, determinare quali recensioni sono positive e quali sono negative, o analizzare il testo delle recensioni alla ricerca di menzioni di entità note come luoghi o persone.

## Visualizza i documenti delle recensioni

Cominciamo a dare un'occhiata ad alcune recensioni di hotel che sono state lasciate dai clienti.

Le recensioni sono in file di testo. Per vederle, esegui semplicemente il codice seguente facendo clic sul pulsante **Esegui cella** (&#9655;) a sinistra della cella.

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Crea una risorsa di Servizi Cognitivi

Per analizzare il testo in queste recensioni, è possibile utilizzare il servizio cognitivo **Analisi del testo**. Per utilizzarlo, devi fornire una risorsa di **Analisi del testo** o di **Servizi cognitivi** nella tua sottoscrizione di Azure (usa una risorsa di Analisi del testo se è l'unico servizio che hai intenzione di usare o se vuoi tenere traccia del suo utilizzo separatamente; altrimenti puoi usare una risorsa di Servizi cognitivi per combinare il servizio di Analisi del testo con altri servizi cognitivi, permettendo agli sviluppatori di usare un singolo endpoint e una singola chiave per accedervi).

Se non ne hai già una, procedi come segue per creare una risorsa di **Servizi Cognitivi** nella tua sottoscrizione di Azure:

> **Nota**: Se disponi già di una risorsa di Servizi Cognitivi, basta aprire la sua pagina **Avvio rapido** nel portale di Azure e copiare la sua chiave e l'endpoint nella cella seguente. Altrimenti, procedi come segue per crearne una.

1. In un'altra scheda del browser, apri il portale di Azure all'indirizzo https://portal.azure.com, accedendo con il tuo account Microsoft.
2. Fai clic sul pulsante **&#65291;Crea una risorsa**, cerca *Servizi cognitivi* e crea una risorsa di **Servizi cognitivi** con le impostazioni seguenti:
    - **Sottoscrizione**: *La tua sottoscrizione di Azure*.
    - **Gruppo di risorse**: *Seleziona o crea un gruppo di risorse con un nome univoco*.
    - **Area geografica**: *Scegli una qualsiasi area disponibile*:
    - **Nome**: *Immetti un nome univoco*.
    - **Piano tariffario**: S0
    - **Confermo di aver letto e compreso gli avvisi**: Selezionato.
3. Attendi il completamento della distribuzione. Vai quindi alla tua risorsa di servizi cognitivi e, nella pagina **Panoramica**, fai clic sul link per gestire le chiavi per il servizio. Avrai bisogno dell'endpoint e delle chiavi per connetterti alla tua risorsa di servizi cognitivi dalle applicazioni client.

### Ottieni la chiave e l'endpoint per la tua risorsa di Servizi cognitivi

Per usare la risorsa di servizi cognitivi, le applicazioni client hanno bisogno del loro endpoint e della chiave di autenticazione:

1. Nel portale di Azure, nella pagina **Chiavi ed endpoint** per la tua risorsa di servizio cognitivo, copia la **Key1** per la tua risorsa e incollala nel codice sottostante, sostituendo **YOUR_COG_KEY**.
2. Copia l'**endpoint** per la tua risorsa e incollalo nel codice sottostante, sostituendo **YOUR_COG_ENDPOINT**.
3. Esegui il codice nella cella seguente facendo clic sul suo pulsante verde <span style="color:green">&#9655;</span>.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## Rileva la lingua
Iniziamo identificando la lingua in cui sono scritte queste recensioni.

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## Estrai le frasi chiave

Ora puoi analizzare il testo nelle recensioni dei clienti per identificare le frasi chiave che danno qualche indicazione dei principali punti di discussione.

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

Le frasi chiave possono aiutarti a capire quali sono i punti di discussione più importanti in ogni recensione. Ad esempio, una recensione che contiene una frase "personale disponibile" o "servizio scadente" può darti un'indicazione di alcune dei principali problemi del recensore.

## Determina la valutazione

Potrebbe essere utile classificare le recensioni come *positive* o *negative* sulla base di un *punteggio di valutazione*. Anche in questo caso, è possibile utilizzare il servizio Analisi del testo per farlo.

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## Estrai entità conosciute

Le *entità* sono componenti che possono essere menzionati nel testo e che si riferiscono a qualche tipo di elemento comunemente compreso, come un luogo, una persona o una data. Supponiamo che tu sia interessato a date e luoghi menzionati nelle recensioni: puoi usare il seguente codice per trovarli.

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

Tieni presente che alcune entità sono sufficientemente note da avere una pagina Wikipedia associata, nel qual caso il servizio di Analisi del testo restituisce l'URL di quella pagina.

## Scopri di più

Per ulteriori informazioni sul servizio Analisi del testo, consulta la [documentazione di Analisi del testo](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)